# a) Función para calcular la transformación SubByte de un Byte

Para calcular la transformación de un byte s<sub>ij</sub> a su correspondiente s<sup>'</sup><sub>ij</sub> hay que seguir una serie de pasos:

1. Primero establecemos los datos sobre los que vamos a trabajar, como lo son el cuerpo, el byte 63 y la longitud de byte.  
  
  
2. Si el byte pasado como parámetro es igual a 0, entonces se devolverá como resultado el byte 63. Si es distinto de 0 se procederá con el algoritmo.
  
  
3. Se calcula la expresión polinomial del byte, transformándolo primero a binario y luego completando el polinomio según si en la posición del coeficiente correspondiente está el bit 1 o no.
  
  
4. Tras esto, calculamos el inverso del byte en el cuerpo (con el simple uso de la operación **pol_byte<sup>-1</sup>**, ya que el cuerpo en el que se trabaja se definió al comienzo.
  
  
5. A continuación, pasamos los coeficientes de dicho polinomio a una lista, la cuál se tratará de la inversa del `byte` pasado como parámetro (si faltan elementos, se rellena con ceros hasta llegar a 8).
  
  
6. Finalmente, se aplica la siguiente fórmula para calcular cada bit de la transformada deseada:
$$b^´_{i} = b_{i} + b_{(i+4)  mód 8} + b_{(i+5)  mód 8} + b_{(i+6)  mód 8} + b_{(i+7)  mód 8} + c_{i}$$  
  
  
7. Se devuelve el byte.

In [4]:
def subByte(byte):
    
    K.<a> = GF(2^8, modulus=x^8 + x^4 + x^3 + x + 1)
    byte_63 = [1, 1, 0, 0, 0, 1, 1, 0]
    byte_length = 8
        
    if byte == "00" or byte == "0x0":
        return "0x63"
    
    binary_b = bin(int(byte, 16))[2:].zfill(byte_length)
    pol_byte = 0*a
    
    for i, bit in enumerate(binary_b):
        if bit == "1":
            pol_byte += a^(len(binary_b)-1-i)
    
    bb = (pol_byte^-1).polynomial()
    
    inverse_byte = []
    for elem in bb:
        inverse_byte.append(elem)
        
    while (len(inverse_byte) < byte_length):
        inverse_byte.append(0)

    subByte = ""
    
    for i, bit in enumerate(inverse_byte):
        bi = (inverse_byte[i] + inverse_byte[(i+4)%byte_length] 
              + inverse_byte[(i+5)%byte_length] + inverse_byte[(i+6)%byte_length] 
              + inverse_byte[(i+7)%byte_length] + byte_63[i])%2
        
        subByte = str(bi) + subByte
        
    return hex(int(subByte, 2))

# b) Función para calcular la matriz de SubBytes

Esta función calcula la matriz de `SubBytes`. Simplemente llama a la función anterior desde el byte 0 hasta el byte 255 y guarda los resultados en una lista de listas.

In [39]:
def subBytesMatrix():
    M = []
    counter = 0
    n_rows_and_cols = 16
    
    for i in range(n_rows_and_cols):
        row = []
        
        for j in range(n_rows_and_cols):
            row.append(subByte(hex(counter)))
            counter += 1
        
        M.append(row)
    
    return M

In [41]:
M = subBytesMatrix();

In [42]:
print(M)

[['0x63', '0x7c', '0x77', '0x7b', '0xf2', '0x6b', '0x6f', '0xc5', '0x30', '0x1', '0x67', '0x2b', '0xfe', '0xd7', '0xab', '0x76'], ['0xca', '0x82', '0xc9', '0x7d', '0xfa', '0x59', '0x47', '0xf0', '0xad', '0xd4', '0xa2', '0xaf', '0x9c', '0xa4', '0x72', '0xc0'], ['0xb7', '0xfd', '0x93', '0x26', '0x36', '0x3f', '0xf7', '0xcc', '0x34', '0xa5', '0xe5', '0xf1', '0x71', '0xd8', '0x31', '0x15'], ['0x4', '0xc7', '0x23', '0xc3', '0x18', '0x96', '0x5', '0x9a', '0x7', '0x12', '0x80', '0xe2', '0xeb', '0x27', '0xb2', '0x75'], ['0x9', '0x83', '0x2c', '0x1a', '0x1b', '0x6e', '0x5a', '0xa0', '0x52', '0x3b', '0xd6', '0xb3', '0x29', '0xe3', '0x2f', '0x84'], ['0x53', '0xd1', '0x0', '0xed', '0x20', '0xfc', '0xb1', '0x5b', '0x6a', '0xcb', '0xbe', '0x39', '0x4a', '0x4c', '0x58', '0xcf'], ['0xd0', '0xef', '0xaa', '0xfb', '0x43', '0x4d', '0x33', '0x85', '0x45', '0xf9', '0x2', '0x7f', '0x50', '0x3c', '0x9f', '0xa8'], ['0x51', '0xa3', '0x40', '0x8f', '0x92', '0x9d', '0x38', '0xf5', '0xbc', '0xb6', '0xda', '0x21',